In [6]:
import pandas as pd
import numpy as np,re
from langchain.schema import HumanMessage, SystemMessage
from langchain.chat_models.gigachat import GigaChat

# Авторизация в сервисе GigaChat
chat = GigaChat(credentials='MzU0M2RmZmMtZTZiYS00NDY2LWE1ODQtZTUzMTFhNTBjZmQ0OjcxYWE1NGY1LWUxZTctNDk1NC1hZGViLTkwOTIyODNiOWVhYw==', verify_ssl_certs=False)

messages = [
    SystemMessage(
        content="Ты лучший аналитик в компании РЖД, необходмо проанализировать данные респондента и вывести вероятность того, уйдет ли респондент от компании и перестанет пользоваться их услугами. Ты даёшь свой ответ только в виде % ухода пользователя и обоснование поставленного процента"
    )
]

In [75]:

class ML_AGEENT:
    def __init__(self) -> None:
        self.dataframe_interesting = pd.read_excel("Интересы.xls")
        self.dataframe_waitings = pd.read_excel("Обращения.xls")
        self.dataframe_transporting = pd.read_excel("Сортированный_Объем Перевозок.xlsx")
        self.dataframe_Vladimyr = pd.read_excel("МС_Владимирская область.xls")
        self.dataframe_Kirov = pd.read_excel("МС_Кировская область.xls")
        self.dataframe_NNov = pd.read_excel("МС_Нижегородская область.xls")
        self.dataframe_MaryEl = pd.read_excel("МС_Республика Марий Эл.xls")
        self.dataframe_Mordoviya = pd.read_excel("МС_Республика Мордовия.xls")
        self.dataframe_Tatarstan = pd.read_excel("МС_Республика Татарстан.xls")
        self.dataframe_Ydmurt = pd.read_excel("МС_Республика Удмуртия.xls")
        self.dataframe_Chyvash = pd.read_excel("МС_Республика Чувашия.xls")
        self.oblast_list = [
                            self.dataframe_Vladimyr, 
                            self.dataframe_Kirov, 
                            self.dataframe_NNov, 
                            self.dataframe_MaryEl, 
                            self.dataframe_Mordoviya, 
                            self.dataframe_Tatarstan,
                            self.dataframe_Ydmurt,
                            self.dataframe_Chyvash
                            ]


    def research(self, id:str) -> list:
        
        _, personal_data = self.research_oblasts(id)
        _, personal_seasons = self.seasons(id)
        _, personal_dop_info,sum = self.research_directions(id)
        persnal_problems = self.research_obrash(id)
        persnal_inters = self.research_interesting(id)
        
        prompt = f"""У нас есть информация о клиенте железнодорожной компании и ключевое значение для анализа:

Значение {sum} - общая сумма платежей клиента компании за последние 9 месяцев. Важно отметить то, что:

Если {sum} > 0, проанализируйте предоставленные данные и оцените вероятность ухода.
Если {sum} == 0, вероятность ухода составляет 100%, и следует предоставить рекомендации для возврата клиента.

Данные о клиенте:

Информация о юридическом лице клиента, платежеспособности и рисках: {personal_data}
Данные о грузоперевозках клиента за последние два года: {personal_seasons}
Информация о взаимодействиях клиента с компанией, таких как обращения за консультированием, поддержка, жалобы, благодарности и т.д.: {persnal_inters}
Данные о проблемах клиента, возникших ранее: {persnal_problems}
Пожалуйста, на основе этих данных, если {sum} > 0, оцените вероятность ухода клиента и предоставьте рекомендации для удержания клиента, если вероятность ухода составляет более 45%. Если {sum} == 0, предложите действия для возвращения клиента с высокой вероятностью ухода (100%)

Сами же данные нужно представить в виде: 

Вероятность ухода: <оценка вероятности ухода в процентах>
Обоснование: <основной текст>"""
        messages.append(HumanMessage(content=prompt))
        res = chat(messages)
        messages.append(res)
        
        pattern = r"Вероятность ухода:\s*(\d+%)\nОбоснование:\s*(.*)"
        
        match = re.search(pattern, res.content, re.DOTALL)

        if match:
            # Извлечение данных из групп регулярного выражения
            churn_probability = match.group(1)  # Вероятность ухода
            justification = match.group(2)  # Обоснование
            
            # Формируем структурированный ответ
            structured_data = {
                "churn_probability": churn_probability,
                "justification": justification
            }
            
            return structured_data
        
        ...
    
    def research_oblasts(self, id:str) -> str:
        for i in self.oblast_list:
            filtred = i[i["ID"] == id][["ID",
                                        "Размер компании.Наименование", 
                                        "Карточка клиента (внешний источник).Индекс платежной дисциплины Описание", 
                                        "Карточка клиента (внешний источник).Индекс финансового риска Описание", 
                                        "Госконтракты.Тип контракта"]]
            if len(filtred.values.tolist()) >=1:
                break
            else:
                pass
        
        all_data = filtred.values.T.tolist()
        # print(all_data[0], all_data[1])
        all_data = [list(set(sublist)) for sublist in all_data]
        # print(all_data[0], all_data[1])
        
        final = "Этот клиент - " +   str(all_data[1]) + " C " + str(all_data[2]) + " С индексом финального риска: " + str(all_data[3]) + " И в гос контрактах он является: " + str(all_data[4])
        # print(final)
        return filtred, final
    
    def seasons(self, id:str) ->  str:
        
        filtred = self.dataframe_transporting[self.dataframe_transporting["ID"] == id]
        years = filtred.columns.values.tolist()[5:]
        month = filtred[years].values.T.tolist()
        month_normal = years[0::2]
        month_in_numbers =  [[sum(subarray)] for subarray in month]
        month_in_tons = month_in_numbers[1::2]
        month_in_money = month_in_numbers[0::2]
        text = ""
        for i in range(len(month_normal)):
            text += f"Клиент в год, месяц - {month_normal[i]} перевёз {month_in_tons[i]} тон груза  на сумму: {month_in_money[i]}\n"
            
        # print(text)
        
        return filtred, text

    def research_directions(self, id:str):
        
        filtred=self.dataframe_transporting[self.dataframe_transporting['ID']==id]
        all_data = filtred.values.T.tolist()
        
        text=""
        sum=0
        
        for i in range(len(all_data[3])):

            text+="| Этот клиент везет из - " + all_data[1][i] + " в - " + all_data[2][i] + "| Код товара - " + str(all_data[3][i])+"| Категория груза - "+all_data[4][i]+"|\n"
            
        # print (text)
        
        years = filtred.columns.values.tolist()[5:]
        month = filtred[years].values.T.tolist()
        
        
        for i in month[-8::2]:
            
            for j in i:
                
                sum+=j
        
        return all_data, text,sum

    def research_interesting(self, id:str) -> str:
        filtred = self.dataframe_interesting[self.dataframe_interesting["ID"] == id][["Состояние", "Канал первичного интереса"]]

        all_data = filtred.values.T.tolist()
        all_data = [list(set(sublist)) for sublist in all_data]

        string = ""
        ls = []

        for i in filtred.values:
            string += str(i[0]) + " " + str(i[1]) + ", "

        for i in all_data[0]:
            for j in all_data[1]:
                if string.count(f"{i} {j}") != 0:
                    ls.append([str(i) + " " + str(j), string.count(f"{i} {j}")])

        for i in ls:
            string += i[0] + " " + str(i[1]) + "\n"

        return string
    

    def research_obrash(self, id:str) -> str:
        filtred = self.dataframe_waitings[self.dataframe_waitings["ID"] == id][["Тема вопроса"]]

        all_data = filtred.values.T.tolist()
        try:
            
            string = ""
            for i in all_data:
                string += i[0] + "\n"

            print(string)
        except:
            string = "Обращений не обнаружено"
        
        return string
 
    def graps_transp(self, id:str) -> list:
        filtred = self.dataframe_transporting[self.dataframe_transporting["ID"] == id]
        years = filtred.columns.values.tolist()[5:][0::2]
        types = list(set(filtred["Гр груза по опер.номен"]))
        tat = []
        for i in types:
            temp = filtred[filtred["Гр груза по опер.номен"] == i]
            data = temp[years].values.T.tolist()
            tat.append([[sum(subarray)] for subarray in data])
            
            
            
        return tat, years, types
        
        
    

In [76]:
agent = ML_AGEENT()

In [78]:
tat, years, types = agent.graps_transp(912)


In [116]:
pipa = []
for i in range(len(tat)):
    tata = []
    for k in range(len(tat[i])):
        tata.append(tat[i][k][0])
    pipa.append(tata)   
pipa[1]

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 26792,
 26792,
 187544,
 53584,
 53673,
 112223,
 0,
 0,
 70540,
 35270,
 153300,
 192478,
 148244,
 157208,
 157208]

In [117]:
import plotly.graph_objects as go
animals=years

fig = go.Figure()
for i in range(len(types)):
    fig.add_trace(go.Bar(name=types[i], x=years, y=pipa[i]),)
fig.update_layout(barmode='group')
fig.show()
